<h2> <center> Tutorial on <br> The Hydrogen atom  </center> </h2>

<br>
<h3> <center> Reinhard J. Maurer </center> </h3>

<div class="col-md-12">
                    <h2> The Hydrogen atom </h2>
                    <div class="col-md-12">
                        <div class="col-md-4">
                            <div class="col-md-10">
                                <img src="hydrogen.jpeg" alt="Motivation">
                            </div>
                        </div>
                        <div class="col-md-8">
                            <p>  Schrödinger equation (SE): $\quad\hat{H}\psi = E\psi $</p>
                            <p class="">  
                            SE: Hamilton ('energy') operator $\hat{H}$ defines the energy and wave function $\psi$  </p>
                            <p class=""> $$ \hat{H} = -\frac{\hbar^2}{2m_e}\nabla^2-\frac{Ze^2}{4\pi\epsilon_0 r}  $$</p>
                            <p> </p>
                            <div class="col-md-12">                            
                            <div class="col-md-12">
                                <b> energies ('eigenvalues') </b>
                                <p> $$ E_{nl} = -\frac{Z^2 m_e e^4}{32\pi^2 \epsilon_0^2 \hbar^2}\frac{1}{n^2}  \quad\text{where}\quad n = 1,2, \dots $$  </p>
                            </div>                        
                            <div class="col-md-12">
                                <b> wave functions ('eigenfunctions')</b> 
                                <p> $$ \psi_{nlm}(r,\theta,\phi) = R_{nl}(r)Y^m_l(\theta,\phi)  $$  </p>
                                <p> $$ R_{nl}(r) = -\Big( \big(\frac{2Z}{na}\big)^3 \frac{(n-l-1)!}{2n[(n+1)!]^3} \Big) \rho^l L^{2l+1}_{n+l}(\rho)\cdot e^{-\rho/2}  $$ </p>
                                <p> $$ \quad\text{with}\quad \rho=(2Z/na)r \quad\text{and}\quad a=4\pi\epsilon_0\hbar^2/m_e e^2  $$</p>
                                <p>   </p>
                            </div>
                            </div>                                
                        </div>
                    </div>
</div>                    


In [5]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
          #'width': 1024,
          #'height': 768,
          'width': 1600,
          'height': 1200,    
})

import numpy as np
from math import sqrt, pi
import warnings
warnings.filterwarnings('ignore')

from bokeh.layouts import gridplot, widgetbox, column
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider
from bokeh.models import TapTool, CustomJS
from bokeh.models import Span
from bokeh.models import Label
from bokeh.models.glyphs import HBar

from bokeh.io import output_notebook, push_notebook, show
from bokeh.palettes import viridis, magma

from ipywidgets import interact

output_notebook()

N=50
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
lw = 2.0
colors = magma(N)
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

{'height': 1200, 'width': 1600}

Loading BokehJS ...

In [6]:
from scipy.special import sph_harm 
from scipy.special import assoc_laguerre
from scipy.misc import factorial

def S(l,m,theta,phi):
    if m>0:
        prefac = ((-1)**m)/sqrt(2.0)
        sph = sph_harm(m,l,theta,phi) + np.conjugate(sph_harm(m,l,theta,phi))
        return prefac*sph        
    elif m<0:
        prefac = ((-1)**m)/(1.j*sqrt(2.0))
        m = abs(m)
        sph = sph_harm(m,l,theta,phi) - np.conjugate(sph_harm(m,l,theta,phi))
        return prefac*sph
    else:
        return sph_harm(0,l,theta,phi)
    
#L_n^k
def L(n,k,x): 
    return assoc_laguerre(x,n,k)

def R(n,l,Z, r):
    prefac = ((2.*Z)**3)*( factorial(n-l-1)/(2*n*factorial(n+l)**3) )
    rho = (2.*Z/n) * r
    return prefac * (rho**l) * L(n+l,2*l+1,rho) * np.exp(-rho/2)
    
    

In [20]:
#define grid
r_max = 20.0
n_r_points = 30
n_theta_points = 30
n_phi_points= 10
r_grid = np.logspace(0,r_max,n_r_points)
theta_grid = np.linspace(0,0.2*pi,n_theta_points)
phi_grid = np.linspace(0,pi,n_phi_points)

Z = 1.0
Ry = 13.60569 #eV
n_max = 4

n_states = 0
for n in range(n_max+1):
    for l in range(0,n):
        for m in range(-l,l+1):
            n_states += 1

#TODO nlm tupel to j index
#TODO j index to nlm tupel

def nlm_2_index(n_ref,l_ref,m_ref):
    i = 0
    for n in range(n_max+1):
        for l in range(0,n):
            for m in range(-l,l+1):
                if n==n_ref and l==l_ref and m==m_ref:
                    index = i
                    break
                i += 1
    return index

def index_2_nlm(index):
    i = 0
    for n in range(n_max+1):
        for l in range(0,n):
            for m in range(-l,l+1):
                if i==index:
                    n_ref = n
                    l_ref = l
                    m_ref = m
                    break
                i += 1
                    
    return (n_ref, l_ref, m_ref)
    

def do_calcs(Z=1):
    E = np.zeros(n_states)
    psi = np.zeros([n_states, n_r_points, n_theta_points, n_phi_points])
    psi_r = np.zeros([n_states, n_r_points])
    for i in range(n_states):
        n, l, m = index_2_nlm(i)
        E[i] = -Z / n**2
        for ri, r in enumerate(r_grid):
            psi_r[i,ri] = R(n,l,Z,r)
            for ti,theta in enumerate(theta_grid):
                for pi, phi in enumerate(phi_grid):
                    psi[i,ri,ti,pi] = R(n,l,Z,r) * S(l,m,theta,phi)
    
    dens_r = psi_r*psi_r
    prob_r = psi_r*psi_r*r*r
    dens = np.conjugate(psi)*psi
    
    return E, psi_r, dens_r, prob_r , psi, dens

def pot(Z,r):
    return -Z/r

In [25]:
Z = 1
E_max = 10.0
E, psi_r, dens_r, prob_r, psi, dens = do_calcs(Z)

#r_grid
#theta_grid
#phi_grid

source1 = ColumnDataSource(data=dict(E=E, right=np.ones(len(E)),color=colors[:len(E)]))
source2 = ColumnDataSource(data=dict(x=r_grid, y=pot(Z,r_grid)))
source3 = ColumnDataSource(data=dict(x=x, y=-psi_r[0,:]))
source4 = ColumnDataSource(data=dict(x=x, y=dens_r[0,:]))

source_vbar = ColumnDataSource(
        data=dict(
            x=[0,L],
        )
    )

callback = CustomJS(code="""""")
tap = TapTool(callback=callback)
#hover = HoverTool(callback=callback)

TOOLS="pan,wheel_zoom,box_zoom,reset"

###########FIRST FIGURE is the Coulomb potential with energy levels and wave functions plotted
#####to the left narrow and tall

################SECOND FIGURE should be the 3d visualization of orbitals

####buttons to switch orbitals
####buttons to switch between orbitals and densities

p1 = figure(x_axis_label='x (Bohr)',y_axis_label='Energy (eV)',
            plot_width=400, plot_height=600, tools=[tap,'pan','box_zoom','wheel_zoom','reset'])
p1.xaxis.axis_label_text_font_size = "16pt"
p1.yaxis.axis_label_text_font_size = "16pt"
p1.xaxis.major_label_text_font_size= "12pt"
p1.yaxis.major_label_text_font_size= "12pt"
#p.extra_y_ranges = {'pot': Range1d(start=-10, end=10000)}
#p.add_layout(LinearAxis(y_range_name='pot'), 'right')
#p.line(x, pib.pot(x), line_width=lw, line_color='grey', y_range_name='pot' )


line_pot = p1.line('x', 'y', source=source2, line_width=lw+1, line_color='grey' )

p1.x_range=Range1d(0.0,r_max)
p1.y_range=Range1d(-20,E_max)

label1 = Label(x=-1, y=E[0], text='n={0:2d}'.format(1),text_color=colors[0],
              text_font_size='14pt')
p1.add_layout(label1)


#p1.xaxis.axis_label = ''
#p1.yaxis.axis_label = ''

energies = p1.hbar(right='right',y='E',height=0.05, left=0, source=source1, color='color')


#p2  = figure(x_axis_label='x (atomic units)',y_axis_label='Wave function', plot_width=400, plot_height=200, 
#             tools=TOOLS)
#p2.xaxis.axis_label_text_font_size = "16pt"
#p2.yaxis.axis_label_text_font_size = "16pt"
#p2.xaxis.major_label_text_font_size= "12pt"
#p2.yaxis.major_label_text_font_size= "12pt"


#vline1=Span(location=0, dimension='height', line_color='grey', line_width=3.0)
#vline2=Span(location=L, dimension='height', line_color='grey', line_width=3.0)
#p2.renderers.extend([vline1,vline2])


#line_psi = p2.line('x','y', source=source3, line_width=lw, line_color='blue')
#vbars1 = p2.vbar('x',top=100, source=source_vbar, width=0.05, bottom=-100, color='grey')

#p2.x_range=Range1d(-3,r_max)
#p2.y_range=Range1d(-0.7,0.7)

#p3  = figure(x_axis_label='x (atomic units)',y_axis_label='Probability', plot_width=400, plot_height=200, 
#             tools=TOOLS)
#p3.xaxis.axis_label_text_font_size = "16pt"
#p3.yaxis.axis_label_text_font_size = "16pt"
#p3.xaxis.major_label_text_font_size= "12pt"
#p3.yaxis.major_label_text_font_size= "12pt"


#vline3=Span(location=0, dimension='height', line_color='grey', line_width=3.0)
#vline4=Span(location=L, dimension='height', line_color='grey', line_width=3.0)
#p3.renderers.extend([vline3,vline4])


#line_dens = p3.line('x','y', source=source4, line_width=lw, line_color='blue')
#vbars2 = p3.vbar('x',top=100, source=source_vbar, width=0.05, bottom=-100, color='grey')
#p3.x_range=Range1d(-3,10)
#p3.y_range=Range1d(-0.1,0.5)

#p_box1 = gridplot([[p1],[p2,p3]])
p_box1 = gridplot([[p1],[]])

#def update(n,m,L):
#    x, E, pot, psi, dens = do_calcs1(m=m,L=L)
#    line_pot.data_source.data['y'] = pot(x)
#    energies.data_source.data['E'] = E
#    energies.data_source.data['right'] = np.ones(len(x))*L
#    #arrow1.line_color=colors[n-1]
#    #arrow1.end.line_color=colors[n-1]
#    label1.y=E[n-1]
#    label1.text='n={0:2d}'.format(n)
#    label1.text_color=colors[n-1]
#    line_psi.data_source.data['y'] = psi[:,n-1]
#    line_psi.glyph.line_color = colors[n-1]
#    line_dens.data_source.data['y'] = dens[:,n-1]
#    line_dens.glyph.line_color = colors[n-1]
#    vbars1.data_source.data['x'] = [0,L]
#    vbars2.data_source.data['x'] = [0,L]
#    push_notebook(handle=h1)

In [24]:
h1=show(p_box1, notebook_handle=True)

W-1002 (EMPTY_LAYOUT): Layout has no children: Row(id='950f06c4-eb45-4b69-b82f-7319a65ca0a2', ...)


In [ ]:
interact(update, n={*range(1,50)}, m=(0.1,1.9), L=(0.1,19.0))